In [2]:
%config IPCompleter.greedy=True

import numpy as np
import matplotlib.pyplot as plt
import sklearn.preprocessing, sklearn.datasets, sklearn.model_selection
import timeit

So far, we talked just about the models. I shown you some plots and I tried to explain some phenomena on them. The results usually seems to be right, however I never talked about the evaluation of the models. How to recognize, if the models is learning and performing well. I would like to correct this in this notebook.

# Loss and metric

First of all, let's talk about losses and metrices. Loss is the function, that the learning algorithm directly optimize. So far we used mean squared error loss (MSE), that is in the form $\frac{1}{n} \sum_{i=0}^{n} \left( t_i - f(\pmb{x_i}) \right)^2$. As $n$ is the constant (we have finite set of examples), optimizing it is the same as optimizing $\frac{1}{2} \sum_{i=0}^{n} \left( t_i - f(\pmb{x_i}) \right)^2$. We computed gradient of this value and optimize it directly. Before moving on, let's remind us the `Neuron` class from the last notebook.

In [2]:
class Neuron:
    def __init__(self, epochs=100, random_state=None, learning_rate=0.001, batch_size=16):
        # use epochs instead of iterations
        self.epochs = epochs
        self.converged = False
        self._rand = np.random.RandomState(random_state)
        self._weights = None
        self._learning_rate = learning_rate
        self._batch_size = batch_size
        pass
    
    def _activation(self, vals):  # activation function
        return 1 / (1 + np.exp(-vals))
    
    def fit(self, X, y, Xtest=None, ytest=None):
        # Initialize the weights
        self.converged = False
        self._weights = self._rand.uniform(-2, 2, X.shape[1])
        n_data = len(X)
        # store gradients and losses
        losses = np.zeros((self.epochs,))
        gradients = np.zeros((self.epochs, int(np.ceil(X.shape[0] / self._batch_size))))
        accuracies = np.zeros((self.epochs,))
        # Learning
        for epoch in range(self.epochs):
            # shuffle the data
            permutation = self._rand.permutation(n_data)
            # for each batch
            for batch_start in range(0, n_data, self._batch_size):
                # get batch
                batch_data = X[permutation[batch_start:batch_start+self._batch_size]]
                batch_target = y[permutation[batch_start:batch_start+self._batch_size]]
                # predict the data
                prediction = self.predict(batch_data)
                # table of gradient for each weights and gradient in the shape (samples,weights)
                gradient = np.reshape(-(batch_target - prediction) * (prediction * (1 - prediction)), newshape=(-1,1)) * batch_data
                # mean gradient over the samples
                op_gradient = self._learning_rate * np.sum(gradient, axis=0)
                # store losses and gradients
                losses[epoch] += np.sum((batch_target - prediction) ** 2, axis=0)
                gradients[epoch, int(batch_start / self._batch_size)] = op_gradient @ op_gradient
                # update the weights
                self._weights = self._weights - op_gradient
                
            # compute accuracy on the test set
            if Xtest is not None and ytest is not None:
                test_prediction = np.where(self.predict(Xtest) < 0.5, 0, 1) 
                accuracies[epoch] = np.sum(test_prediction == ytest) / len(ytest)
                
        return losses / X.shape[0], gradients, accuracies
    
    def predict(self, X):
            return self._activation(X @ self._weights)
    
    def __call__(self, X):
        return self.predict(X)

The `fit` method return losses, gradients and accuracies over the epochs. As we can see from the code, we compute gradient for each example and update weights. In the end, the method returns `losses / X.shape[0]`, that is the mean loss over the whole dataset.

There is one important aspect to understand here - loss is independent between the examples. As the loss is the value, that is directly optimized, it should always decrease. If we had only single example and use this algorithm, it needs to decrease every step (unless the loss is zero). However, as optimizing loss for one example can increase the loss of other example (and loss is  independent for each exaple), the loss of the whole dataset (mean of losses for each example) can increase.

How can we make sure this doesn't happen? We may optimize the dataset loss instead of loss per each example - that is exactly what the batch gradient descent do! Although it still computes the loss for each example, there is only one update, that takes into account all the examples. As such, it optimize the dataset loss and the loss never increase (as you can see in the end of the previous notebook). As you may guess, using minibatch gradient descent we try to use this property using less examples.

Through the following notebooks we are going to see a few more losses. They usage depends on the task we are doing - for regresion tasks (we try to predict continuous value like age, weights and so on) we use MSE loss. For classification tasks (we try to predict in which class the example is from) we usually use cross-entropy loss function. Note that we were doing only classification so far, and we still used MSE loss. It was good enought for out purposes so far, but we will talk about cross-entropy soon.

Now let's talk about metrices. Metric, unlike loss, is not directly optimized during training. Decrease in loss should result in better metric. In the last notebook we have seen accuracy - how many examples are classified correctly. I will use accuracy to demonstrate some interesting aspects.

Unlike loss, the metric doesn't need to decrease. Especially for accuracy, the better the model, the higher accuracy you will get. The accuracy should take more sense for you are as a programmer (or as data scientist) rather than for a computer. It may decrease, increase, oscilate around some value and so on. Its up to you to interpret it.

Secondly, metric is not independent between examples. Compute accuracy for subset of dataset doesn't make much sense. Although we may think about accuracy as mean of values 0 (when the prediction was wrong) and 1 (when the prediction was correct), it doesnt need to be this way. You may see some matrices on the [tensorflow documentation page](https://www.tensorflow.org/api_docs/python/tf/keras/metrics). There are metrices like false positive, hinge, recall and so on. The important fact is that metric, unlike loss, is some value that is aggregated over the whole dataset, rather than computed independently for each example.

And lastly, metric is optimized indirectly. In our example, we predicted class $0$ if the value was $<0.5$ and $1$ otherwise. Take for example, that our model returned $0.7$ and we corrected it, so it returns $0.6$ (it is the class $0$). In this case, the loss decreased, but the accuracy remained same, as the prediction is still class $1$, although it has lower probability. This should demonstrace you, that not only loss, but why the metrices are important as well. Usually, when you have task in hand, you are not interested in the model loss, but rather in some of it's metric. 

# Train set, development set and test set

We know what to measure, but we still don't know how to measure it. When we use the whole dataset for training, we dont know the real performance of the model. The problem is overfitting or underfitting, terms that I will explain in a moment. Our goal is not to get best score on the dataset we use for training, but rather best score on unseen data - something called generalization error. You may imagine it as if the model has a memory - weights of neuron is some kind of memory. If the model can remember all the training examples and their respective target value, the error on the training set would be zero. However, when we pass examples that we didn't use for training, the prediction can be completely wrong. The problem is that the model is too much aligned with the training data and do not generalize well.

So how to get the real performance of the model? Just use the unseen data to evaluate the performance of the model. Split dataset into **training set** and **test set**, train the model on the training set and evaluate it on the test set. The ratio is usually something between $0.7:0.3$ to $0.8:0.2$. That is exactly what I did in the previous notebooks, if you read the code carefully. I used [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function from `sklearn.model_selection` namespace. You pass there array you wanna split and ratio.

In [3]:
data, target = sklearn.datasets.fetch_openml('mnist_784', version=1, return_X_y=True)
train_data, test_data, train_target, test_target = sklearn.model_selection.train_test_split(data, target, test_size=0.25, random_state=47)

In [4]:
print(len(train_data), len(test_data))
print(len(train_target), len(test_target))

52500 17500
52500 17500


Now we have train data and test data and we may measure the performance of the model. There is however one drawback - the model has some parameters, that we need to set up. For us this can be learning rate, batch size or number of epochs. These are not parameters that are learned during training but we need to set them up in advance - they are called **hyperpatameters**. No one is so skilled that he could guess the parameters correctly, there is usually some experimenting and searching for those parameters. We may use the test set and pick up the combination of hyperparameters, that has lowest test error. But that brings us back where we were before - how to measure the real performance of the model. We used the test set to set up hyperparameters, so obviously it's the best model that perform on the test set. But what about unseen data, is it still the case?

What seems to be a good idea is split data into three parts - **training set** that we use for training and we use it to set up trainable parameters (like weights). Then we use **development set** for hyperparameter search - based on the performance on the development set we pick up the best model. Finally, we use **test set** to evaluate the performance of the model on unseen data.

This approach has one drawback - we reduced the amount of training data significantly. From the whole dataset, we use only a fraction for the training - usually something between 60% and 70%. Still it seems this approach has the best performance estimation so far. In a moment I will talk about *K-fold cross validation*, that doesn't use validation set per se, but is computably much more intensive.

There is no build-in function to split data into training, validation and test set. However we may use `train_test_split` twice. FOr example to split data in ratio $0.6:0.2:0.2$ we may use following code.

In [8]:
train_data, devtest_data, train_target, devtest_target = sklearn.model_selection.train_test_split(data, target, test_size=0.4, random_state=47)
dev_data, test_data, dev_target, test_target = sklearn.model_selection.train_test_split(devtest_data, devtest_target, test_size=0.5, random_state=47)
print(len(train_data), len(dev_data), len(test_data))
print(len(train_target), len(dev_target), len(test_target))

42000 14000 14000
42000 14000 14000


# Underfitting, overfitting and model capacity

